In [1]:
import pandas as pd

In [2]:
def one_hot_list(data, column, prefix=None):
    data[column] = data[column].str.split(", ")
    data = data.explode(column)
    data_enc = pd.get_dummies(data[column], prefix=prefix)
    encoded_data = pd.concat([data, data_enc], axis=1)
    return encoded_data

# Courses compulsory for each degree

In [3]:
enrol_nums_data = pd.read_excel("Raw Data/Course Enrollment Numbers.xlsx", sheet_name="abbreviated course names")

In [4]:
enrol_nums_data.to_excel("Processed Data/web_scraped_data.xlsx", index = False)

In [59]:
compulsory_enc = one_hot_list(enrol_nums_data, "Degrees Compulsory For")
# compulsory_enc.to_excel("compulsory_encoded.xlsx", index = False)

In [60]:
list(compulsory_enc.columns)[11:]

['ambsc',
 'ammmath',
 'cammsc',
 'cmfft',
 'cmfpt',
 'fmoft',
 'fmopt',
 'mbsc',
 'mma',
 'mmmath',
 'mnbi',
 'mnbu',
 'mnmu',
 'mnph',
 'mnst',
 'orco',
 'ords',
 'ormsc',
 'orwr',
 'stds',
 'storft',
 'storpt']

In [61]:
comp_ambsc = compulsory_enc[compulsory_enc["ambsc"] == True] 
comp_ammmath = compulsory_enc[compulsory_enc["ammmath"] == True]
comp_cammsc = compulsory_enc[compulsory_enc["cammsc"] == True]
comp_cmfft = compulsory_enc[compulsory_enc["cmfft"] == True]
comp_cmfpt = compulsory_enc[compulsory_enc["cmfpt"] == True]
comp_fmoft = compulsory_enc[compulsory_enc["fmoft"] == True] 
comp_fmopt = compulsory_enc[compulsory_enc["fmopt"] == True]
comp_mbsc = compulsory_enc[compulsory_enc["mbsc"] == True]
comp_mma = compulsory_enc[compulsory_enc["mma"] == True]
comp_mmmath = compulsory_enc[compulsory_enc["mmmath"] == True]
comp_mnbi = compulsory_enc[compulsory_enc["mnbi"] == True] 
comp_mnbu = compulsory_enc[compulsory_enc["mnbu"] == True]
comp_mnmu = compulsory_enc[compulsory_enc["mnmu"] == True]  
comp_mnph = compulsory_enc[compulsory_enc["mnph"] == True] 
comp_mnst = compulsory_enc[compulsory_enc["mnst"] == True]
comp_orco = compulsory_enc[compulsory_enc["orco"] == True] 
comp_ords = compulsory_enc[compulsory_enc["ords"] == True]
comp_ormsc = compulsory_enc[compulsory_enc["ormsc"] == True]
comp_orwr = compulsory_enc[compulsory_enc["orwr"] == True]
comp_stds = compulsory_enc[compulsory_enc["stds"] == True]
comp_storft = compulsory_enc[compulsory_enc["storft"] == True]  
comp_storpt = compulsory_enc[compulsory_enc["storpt"] == True] 

# Courses within each category

In [27]:
category_enc = one_hot_list(enrol_nums_data, "Categories")

In [28]:
list(category_enc.columns)[11:]

['Algebra',
 'Analysis',
 'Applied & Computational',
 'Data Science',
 'Financial',
 'Geometry & Topology',
 'Mathematical Physics',
 'Optimization & Operational Research',
 'Probability',
 'Statistics']

In [31]:
algebra_courses = category_enc[category_enc["Algebra"] == True]
analysis_courses = category_enc[category_enc["Analysis"] == True]
app_comp_courses = category_enc[category_enc["Applied & Computational"] == True]
data_science_courses = category_enc[category_enc["Data Science"] == True]
financial_courses = category_enc[category_enc["Financial"] == True]
geom_top_courses = category_enc[category_enc["Geometry & Topology"] == True]
mat_phys_courses = category_enc[category_enc["Mathematical Physics"] == True]
opt_or_courses = category_enc[category_enc["Optimization & Operational Research"] == True]
probability_courses = category_enc[category_enc["Probability"] == True]
statistics_courses = category_enc[category_enc["Statistics"] == True]

# Testing

In [5]:
enc_tt_data = pd.read_excel("Processed Data/encoded_timetable_data.xlsx")

In [48]:
enc_tt_grouped = enc_tt_data.sort_values(by=["ws_groups"])